<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br /><span xmlns:dct="http://purl.org/dc/terms/" property="dct:title"><b>Misc exercises</b></span> by <a xmlns:cc="http://creativecommons.org/ns#" href="http://mate.unipv.it/gualandi" property="cc:attributionName" rel="cc:attributionURL">Stefano Gualandi</a> is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.<br />Based on a work at <a xmlns:dct="http://purl.org/dc/terms/" href="https://github.com/mathcoding/opt4ds" rel="dct:source">https://github.com/mathcoding/opt4ds</a>.

**NOTE:** Run the following script whenever running this script on a Google Colab.

In [ ]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("glpk") or os.path.isfile("glpk")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq glpk-utils
    else:
        try:
            !conda install -c conda-forge glpk 
        except:
            pass

# Domande di Python, esempi di modelli in Pyomo

## 1. Domande ricevute
### Domanda 1: Il problema di *aliasing* nelle liste
Le liste in Python sono delle strutture dati *mutable*, ovvero che possono essere modificate, a differenze delle tuple che sono una struttura dati *read only*, ovvero che una volta che sono inizializzate non possono essere più modificate.

Quando si usano le liste, insieme all'operatore di assegnamento `=` (da non confondere con l'operatore logico di confronto `==`), quello che succedde è che non viene copiata la lista in una nuova variabile, ma viene creato un *handle* (un puntatore) alla stessa lista. Vediamo un esempio concreto di seguito.

In [1]:
As = [1, 2, 3, 4]
Bs = As

Bs.pop(2)

print("As:", As)
print("Bs:", Bs)

As: [1, 2, 4]
Bs: [1, 2, 4]


Si noti che a tutti gli effetti, `Bs` reindrezza l'operazione `pop` alla struttura dati a cui si riferesce `As`. Usando una notazione **box-and-pointer**, quello che succede dovrebbe essere subito chiaro.

Per maggiori dettagli, si veda il paragrafo 4.4.1 del notebook seguente:
[Link al corso di Python](http://www-dimat.unipv.it/gualandi/programming/notebooks/Lab11_TupleListeDizionari.html)

Se uno avesse veramente voluto creare una copia della lista `As` in `Bs`, avrebbe potuto usare il costruttore delle list, o semplicemente un operatore di slicing, come nell'esempio seguente.

In [2]:
As = [1, 2, 3, 4, 5]
Bs = list(As)
Cs = Bs[0:len(As)]

Bs.pop(2)
Cs.pop(3)

print("As:", As)
print("Bs:", Bs)
print("Cs:", Cs)

As: [1, 2, 3, 4, 5]
Bs: [1, 2, 4, 5]
Cs: [1, 2, 3, 5]


### Domanda 2: Differenza tra modello e programma
Domanda ricevuta:

> [...] l'idea che mi sono fatto è che, in questo caso, sia sbagliato il modello di partenza, l'idea con cui intendo risolvere il problema. Ma poi mi chiedo: siccome non è la prima volta che riscontro un problema di questo tipo con pyomo, io non posso in alcun modo definire nuove variabili a partire dal valore che altre variabili assumono? 

La difficoltà principale nell'esercitazione sul CVRP è riuscire a mettere a fuoco la differenza tra l'implementare un **algoritmo** che risolve un problema, e invece tra troverro un **modello di programmazione lineare intera** che formalizza un problema usando solo

1. Variabili decisionali continue o intere
2. Funzione obiettivo lineare
3. Vincoli lineari

In particolare, sulla parte di domanda:

> non posso in alcun modo definire nuove variabili a partire dal valore che altre variabili assumono? 

La risposta è *yes, you can!*, ma devi riuscire a scrivere la relazione che vuoi esprimere tra le variabili in termini di vincoli lineari. Non devi implementare un algoritmo che calcola quella relazione.

Una volta scritto il modello, sarà un risolutore generico di programmazione lineare intera ad eseguire i suoi algoritmi, per cercare di risolvere il problema, e dichiarare il problema come (e dovete controllare lo status):

1. Non ammissibile
2. Illimitato
3. Risolto all'ottimo. 
4. Oppure: *raggiunto un time limit, o un raggiunto iteration limit, o esaurita le memoria...)

### Domanda 3: Variabili, parametri, assegnamento iniziale
Consideriamo il codice seguente:

In [3]:
# Dati letti dal file:
Xs = [(266, 235), (295, 272), (301, 258), (309, 260), (217, 274), (218, 278), (282, 267), (242, 249), (230, 262), (249, 268), (256, 267), (265, 257), (267, 242), (259, 265), (315, 233), (329, 252), (318, 252), (329, 224), (267, 213), (275, 192), (303, 201), (208, 217), (326, 181)]
Ws = [0, 125, 84, 60, 500, 300, 175, 350, 150, 1100, 4100, 225, 300, 250, 500, 150, 100, 250, 120, 600, 500, 175, 75]

# Funzione distanza
from math import sqrt
def Distance(A, B):
    return sqrt((A[0]-B[0])**2 + (A[1] - B[1])**2)

In [13]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, ConstraintList
from pyomo.environ import Binary, RangeSet, NonNegativeReals, NonNegativeIntegers, Reals

n = len(Xs)
C = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        C[i,j] = Distance(Xs[i],Xs[j])
        
        
model = ConcreteModel()

model.I = RangeSet(len(Xs))
model.J = RangeSet(len(Ws))
  
model.x = Var(model.I, model.J, within=Binary)

#model.c = Var(model.I, model.J, within=NonNegativeReals)

model.vincoli = ConstraintList()
for i in model.I:
    for j in model.J:
        if j is not i:
            model.vincoli.add(expr = model.x[i,j] == 0)
            #model.x[i,j]() = 0
            #model.c[i,j]() = Distance(Xs[i-1],Xs[j-1]) 
            
model.obj = Objective(expr = sum(C[i-1,j-1]*model.x[i,j] for i in model.I for j in model.J) )

In [11]:
import numpy as np

n = len(Xs)
C = np.zeros((n,n))
for i in range(n):
    for j in range(n):
        C[i,j] = Distance(Xs[i],Xs[j])
        
print(C[2,3], C[3,2], C[3,3])

15.231546211727817 15.231546211727817 0.0


## 2. Modelli in Pyomo
Di seguito vediamo come sia possibile scrivere alcuni modelli visti a lezione in Pyomo.

### 2.1 Problema di programmazione lineare casuale
Consideriamo la funzione seguente che crea un problema di Programmazione Lineare Intera (PLI) casuale.

Un problema di PLI è definito dai due vettori $c$ e $b$ e dalla matrice $A$:

$$
    z = \min \,\{ c x \mid Ax \geq b, x \geq 0 \,\}
$$

Scriviamo prima una funzione per generare i dati in modo casuale.

In [14]:
from numpy import random
def RandomILP(n, m, seed=13):
    random.seed(seed)
    c = random.randint(1, 10, size=n)
    b = random.random(m)
    A = random.randint(1, 10, size=(m,n))
    
    return c, b, A
    
print(RandomILP(2,3))

(array([3, 1]), array([0.23754122, 0.82427853, 0.9657492 ]), array([[4, 5],
       [3, 7],
       [6, 5]]))


Scriviamo ora una funzione che presi in input i dati di un'istanza, costruisce il modello, lo risolve, controlla lo status, e restituisce un eventuale soluzione.

In [33]:
from pyomo.environ import maximize

def SolveILP(c, b, A):
    m = len(b)
    n = len(c)
    
    model = ConcreteModel()
    
    model.I = RangeSet(n)
    model.J = RangeSet(m)
    
    model.x = Var(model.I, within=NonNegativeIntegers)
    
    model.obj = Objective(expr = sum(c[i-1]*model.x[i] for i in model.I))
#                          sense = maximize)
    
    model.vincoli = ConstraintList()
    for j in model.J:
        model.vincoli.add(expr = sum(A[j-1, i-1]*model.x[i] for i in model.I) >= b[j-1])
    
    sol = SolverFactory('glpk').solve(model, tee=False)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.I]
    
    return 'optimal', model.obj(), xbar

In [34]:
c, b, A = RandomILP(2, 3, 1717)
print(c,b,A)
print(SolveILP(c, b, A))

[2 1] [0.61339435 0.86048185 0.72666267] [[2 1]
 [3 9]
 [9 6]]

Status: ok
Termination condition: optimal
Statistics: 
  Branch and bound: 
    Number of bounded subproblems: 3
    Number of created subproblems: 3
Error rc: 0
Time: 0.15018582344055176

('optimal', 1.0, [0.0, 1.0])


### 2.2 Turnistica infermieri
Vediamo ora come possiamo scrivere il modello che rappresenta i turni settimanali degli infermieri del San Matteo di Pavia.

Per la descrizione del modello si vedano le slides del corso.


In [48]:
def StaffScheduling():
    # Data
    d = 7
    w = 5
    demand = [5,6,7,3,3,2,2]
    cost = [1.0, 1.0, 1.01, 1.02, 1.03, 1.07, 1.05]
    
    A = np.zeros( (d,d) )
    for i in range(d):
        for j in range(w):
            A[(i+j)%d, i] = 1
    
    model = ConcreteModel()
    
    model.Day = RangeSet(d)
    
    model.x = Var(model.Day, within = NonNegativeIntegers)
    
    model.obj = Objective(expr=sum(cost[i-1]*model.x[i] for i in model.Day))
    
    model.cover = ConstraintList()
    for day in model.Day:
        model.cover.add(expr=sum(A[day-1, i-1]*model.x[i] for i in model.Day) >= demand[day-1])
    
    model.write("staff.lp")
   
    sol = SolverFactory('glpk').solve(model, tee=True)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.Day]
    
    return 'optimal', model.obj(), xbar
 

In [49]:
print(StaffScheduling())

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\gualandi\AppData\Local\Temp\tmpucro1uz6.glpk.raw --wglp
 C:\Users\gualandi\AppData\Local\Temp\tmpg34qvej1.glpk.glp --cpxlp C:\Users\gualandi\AppData\Local\Temp\tmp_6iny12i.pyomo.lp
Reading problem data from 'C:\Users\gualandi\AppData\Local\Temp\tmp_6iny12i.pyomo.lp'...
8 rows, 8 columns, 36 non-zeros
7 integer variables, none of which are binary
90 lines were read
Writing problem data to 'C:\Users\gualandi\AppData\Local\Temp\tmpg34qvej1.glpk.glp'...
78 lines were written
GLPK Integer Optimizer, v4.65
8 rows, 8 columns, 36 non-zeros
7 integer variables, none of which are binary
Preprocessing...
7 rows, 7 columns, 35 non-zeros
7 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7
Solving LP relaxation...
GLPK Simplex Optimizer, v

In [43]:
!more staff.lp

\* Source Pyomo model name=unknown *\

min 
x8:
+1 x1
+1 x2
+1 x3
+1 x4
+1 x5
+1 x6
+1 x7

s.t.

c_l_x9_:
+1 x1
+1 x4
+1 x5
+1 x6
+1 x7
>= 5

c_l_x10_:
+1 x1
+1 x2
+1 x5
+1 x6
+1 x7
>= 6

c_l_x11_:
+1 x1
+1 x2
+1 x3
+1 x6
+1 x7
>= 7

c_l_x12_:
+1 x1
+1 x2
+1 x3
+1 x4
+1 x7
>= 3

c_l_x13_:
+1 x1
+1 x2
+1 x3
+1 x4
+1 x5
>= 3

c_l_x14_:
+1 x2
+1 x3
+1 x4
+1 x5
+1 x6
>= 2

c_l_x15_:
+1 x3
+1 x4
+1 x5
+1 x6
+1 x7
>= 2

c_e_ONE_VAR_CONSTANT: 
ONE_VAR_CONSTANT = 1.0

bounds
   0 <= x1 <= +inf
   0 <= x2 <= +inf
   0 <= x3 <= +inf
   0 <= x4 <= +inf
   0 <= x5 <= +inf
   0 <= x6 <= +inf
   0 <= x7 <= +inf
general
  x1
  x2
  x3
  x4
  x5
  x6
  x7
end


### 2.3 RNA folding
Si consideri il modello visto a lezione nelle slides `RNA folding`, vediamo come implementare quel modello in Pyomo.

In [57]:
from pyomo.environ import maximize

def RNAfold(s):
    C1 = ('A','U')
    C2 = ('C','G')
    
    ns = len(s)
    
    E = []
    for i in range(ns):
        for j in range(ns):
            if i < j:
                E.append( (i,j) )
                
    n = len(E)
    
    # Model
    model = ConcreteModel()
    model.P = RangeSet(n)
    model.x = Var(model.P, within=Binary)
    
    model.obj = Objective(expr = sum(model.x[p] for p in model.P),
                          sense = maximize)
    
    # Vincoli
    model.setzero = ConstraintList()
    for p in model.P:
        i,j = E[p-1]
        if not((s[i],s[j]) == C1 or (s[j],s[i]) == C1 or (s[i],s[j]) == C2 or (s[j],s[i]) == C2):
            model.setzero.add(expr = model.x[p] == 0)
    
    model.atmost = ConstraintList()
    for j in range(ns):
        L1 = []
        L2 = []
        for p in model.P:
            v,w = E[p-1]
            if j == w:
                L1.append( p )
            if j == v:
                L2.append( p )
        Ls = L1 + L2
        model.atmost.add(expr=sum(model.x[p] for p in Ls) <= 1)
    
    model.conf = ConstraintList()
    for p in model.P:
        i, j = E[p-1]
        for q in model.P:
            v, w = E[q-1]
            if i < v < j < w:
                model.conf.add(expr= model.x[p] + model.x[q] <= 1)
    
    sol = SolverFactory('glpk').solve(model, tee=True)
    
    for info in sol['Solver']:
        print(info)
        
    xbar = [model.x[i]() for i in model.P]
    
    return 'optimal', model.obj(), xbar

In [58]:
s = 'ACGUGCCCGAU'
print(RNAfold(s))

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\gualandi\AppData\Local\Temp\tmptnyy8yso.glpk.raw --wglp
 C:\Users\gualandi\AppData\Local\Temp\tmprbk2d_jz.glpk.glp --cpxlp C:\Users\gualandi\AppData\Local\Temp\tmpqqjmjyy4.pyomo.lp
Reading problem data from 'C:\Users\gualandi\AppData\Local\Temp\tmpqqjmjyy4.pyomo.lp'...
C:\Users\gualandi\AppData\Local\Temp\tmpqqjmjyy4.pyomo.lp:2072: warning: lower bound of variable 'x1' redefined
C:\Users\gualandi\AppData\Local\Temp\tmpqqjmjyy4.pyomo.lp:2072: warning: upper bound of variable 'x1' redefined
381 rows, 56 columns, 810 non-zeros
55 integer variables, all of which are binary
2127 lines were read
Writing problem data to 'C:\Users\gualandi\AppData\Local\Temp\tmprbk2d_jz.glpk.glp'...
1648 lines were written
GLPK Integer Optimizer, v4.65
381 rows, 56 columns, 810 non-zeros
55 integer variables, all of which are binary
Preprocessing...
33 rows, 16 columns, 76 non-zeros
16 integer variables, all of whic